In [ ]:
import glob
import importlib
import numpy as np
import os
import time

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import nitelite_mapmaker

In [ ]:
importlib.reload(nitelite_mapmaker)

# Settings

In [ ]:
flight_name = '220513-FH135'
data_dir = '/Users/Shared/data'
google_drive_dir = '/Users/zhafensaavedra/Google Drive'

In [ ]:
# Directory containing manually-georeferenced data
manually_reffed_dir = '/Users/zhafensaavedra/Google Drive/Shared drives/NITELite/Data & Analysis/QGIS FH135/FH135 Main Project/Main Geo Files'
# If set to "None", one is chosen randomly.
manually_reffed_fn = 'Geo 959732303_0.tif'

In [ ]:
image_dir = os.path.join(data_dir, 'nitelite/images', flight_name, '23085686' )
metadata_dir = os.path.join(google_drive_dir, 'Shared drives/NITELite/Data & Analysis/Old NITELite Flights', flight_name, 'data')
img_log_fp = os.path.join(metadata_dir, 'image.log')
imu_log_fp = os.path.join(metadata_dir, 'OBC/PresIMULog.csv')
gps_log_fp = os.path.join(metadata_dir, 'OBC/GPSLog.csv')

In [ ]:
rng = np.random.default_rng(1425)

# Resampling

## Setup

In [ ]:
mm = nitelite_mapmaker.mapmaker.Mapmaker(
    image_dir=image_dir,
    img_log_fp=img_log_fp,
    imu_log_fp=imu_log_fp,
    gps_log_fp=gps_log_fp,
)

In [ ]:
greffer = nitelite_mapmaker.georeference.GeoReferencer()

## Set Up Originals

In [ ]:
imgs = {}

In [ ]:
def get_rotated_coords(xs_img_frame, ys_img_frame, theta=np.pi/4., ll_coords=np.array([1., 2.])):
    
    xs_img_frame_mesh, ys_img_frame_mesh = np.meshgrid(xs_img_frame, ys_img_frame)
                       
    xs = ll_coords[0] + xs_img_frame_mesh * np.cos(theta) + ys_img_frame_mesh * np.sin(theta)
    ys = ll_coords[1] + xs_img_frame_mesh * np.sin(-theta) + ys_img_frame_mesh * np.cos(theta)
    
    return xs, ys

### Random uniform

In [ ]:
uniform_img = rng.uniform(size=(1200,1920))

In [ ]:
xs_img_frame = np.linspace(0., 10., uniform_img.shape[1])
ys_img_frame = np.linspace(0., 10., uniform_img.shape[0])

In [ ]:
imgs['uniform'] = {
    'points': get_rotated_coords(xs_img_frame, ys_img_frame),
    'img': uniform_img,
}

### Orientation

In [ ]:
orientation_img = np.array([
    [ 1., 1., 1., 0., 0., 0., 0., ],
    [ 1., 1., 1., 0., 0., 0., 0., ],
    [ 0., 0., 0., 0., 0., 0., 0., ],
    [ 0., 0., 0., 0., 0., 0., 0., ],
    [ 0., 0., 0., 0., 0., 0., 0., ],
    [ 0., 0., 0., 0., 0., 0., 0., ],
    [ 0., 0., 0., 0., 0., 0., 0., ],
])

In [ ]:
xs_img_frame = np.linspace(0., 10., orientation_img.shape[1])
ys_img_frame = np.linspace(0., 10., orientation_img.shape[0])

In [ ]:
imgs['orientation'] = {
    'points': get_rotated_coords(xs_img_frame, ys_img_frame),
    'img': orientation_img,
}

### Actual random image

In [ ]:
fp = rng.choice(glob.glob(os.path.join(image_dir, '*.raw')))

In [ ]:
img = mm.flight.get_rgb_img(fp)[:,:,1]

In [ ]:
xs_img_frame = np.arange(img.shape[1])
ys_img_frame = np.arange(img.shape[0])

In [ ]:
imgs['actual'] = {
    'points': get_rotated_coords(xs_img_frame, ys_img_frame),
    'img': img,
}

## Histogram Method

In [ ]:
methods = list(imgs.keys())

In [ ]:
mosaic = [['original_{}'.format(_), 'resampled_{}'.format(_)] for _ in methods]

In [ ]:
fig = plt.figure(figsize=(len(mosaic[0])*10,len(mosaic)*10))

ax_dict = fig.subplot_mosaic(mosaic)

for method in methods:
    
    start = time.time()
    
    (xs_resampled, ys_resampled), img_resampled = greffer.resample(**imgs[method])
    
    time_taken = time.time() - start
    print('{} took {:.2g} seconds'.format(method, time_taken))
    
    ax = ax_dict['original_{}'.format(method)]
    ax.pcolormesh(
        imgs[method]['points'][0],
        imgs[method]['points'][1],
        imgs[method]['img'],
        vmin=0,
        vmax=1,
    )
    ax.set_aspect('equal')

    ax = ax_dict['resampled_{}'.format(method)]
    ax.pcolormesh(
        xs_resampled,
        ys_resampled,
        img_resampled,
        vmin=0,
        vmax=1,
    )
    ax.set_aspect('equal')

Looks pretty clear that the histogram method introduces some artifacts.